## 参考  
### kaggle notebook lightGBM
https://www.kaggle.com/girmdshinsei/for-japanese-beginner-with-wrmsse-in-lgbm?scriptVersionId=31044557

### lightGBM 時系列
https://qiita.com/ground0state/items/657861de619a4e4a30de

### Feature Scalingはなぜ必要？ 標準化と正規化の使い分け
https://qiita.com/ttskng/items/2a33c1ca925e4501e609

### Quick start catboost
https://catboost.ai/docs/concepts/python-quickstart.html

### Pythonでcatboostを使ってみる
#### (cat_featuresの使い方を調べた)
https://qiita.com/shin_mura/items/3d9ce25a60bdd25a3333

### LightGBMとOptunaを導入・動かしてみる
https://kiseno-log.com/2019/11/05/lightgbm%E3%81%A8optuna%E3%82%92%E5%B0%8E%E5%85%A5%E3%83%BB%E5%8B%95%E3%81%8B%E3%81%97%E3%81%A6%E3%81%BF%E3%82%8B/

In [1]:
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt

# ラベルエンコーダー
from sklearn import preprocessing, metrics

# lightGBM
import lightgbm as lgb

# CatBoost
from catboost import Pool, CatBoostRegressor

# XGBoost
import xgboost as xgb

# 1セルでまとめて.head()、.tail()等を入力しても大丈夫になる
from IPython.display import display

# 標準化
from sklearn.preprocessing import StandardScaler

# 誤差算定
from sklearn.metrics import mean_absolute_error

# 二乗平均平方根誤差 (RMSE)
from sklearn.metrics import mean_squared_error

# 決定係数
from sklearn.metrics import r2_score

import optuna.integration.lightgbm as lgb_o

import warnings
warnings.filterwarnings('ignore')

# DataFrameの表示数を変更
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

### データ作成

In [2]:
# ローカル用
path = os.getcwd() + "/"

# kaggle Notebook用
INPUT_DIR = '../input/m5-forecasting-accuracy'

calendar.csv -製品の販売日に関する情報が含まれています。
sales_train_validation.csv -製品および店舗ごとの過去の毎日の販売台数データが​​含まれています [d_1 - d_1913]
sample_submission.csv-提出の正しい形式。詳細については、「評価」タブを参照してください。
sell_prices.csv -店舗および日付ごとに販売された製品の価格に関する情報が含まれています。

sales_train_evaluation.csv-締め切りの1か月前に1回ご利用いただけます。売上高が含まれます[d_1 - d_1941]

各行は含むidの連結であるitem_idとstore_idのいずれかである、validation（公共のランキングに対応する）、またはevaluation（プライベートランキングに対応します）。　　 F1-F28各行で販売されるアイテムの28予測日（）を予測しています。　　 以下のためのvalidation行、これに相当するd_1914 - d_1941、とのためevaluationの行、これに相当しますd_1942 - d_1969。　　

validation d_1914 - d_1941の単価と量を予測する。１か月前に１回公開される。 evaluation d_1942-d_1969の単価と量を予測する

In [ ]:
# sales_train_validation.csv
try:
    stv = pd.read_csv(path + "sales_train_validation.csv") # ローカル用
except FileNotFoundError:
    stv = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv") # kaggle用

    
# calendar.csv
try:
    cal = pd.read_csv(path + "calendar.csv") # ローカル用
except FileNotFoundError:
    cal = pd.read_csv(f"{INPUT_DIR}/calendar.csv") # kaggle用

    
# sell_prices.csv
try:
    price = pd.read_csv(path + "sell_prices.csv") # ローカル用
except FileNotFoundError:
    price = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv") # kaggle用

    
# sample_submission.csv
try:
    ss = pd.read_csv(path + "sample_submission.csv") # ローカル用
except FileNotFoundError:
    ss = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv") # kaggle用

stv.shape

### ロースペックマシン限定

In [3]:
stv = pd.read_csv(path + "sales_train_validation.csv",
                               skiprows=lambda x: x not in range(0,1001))

cal = pd.read_csv(path + "calendar.csv",
                               skiprows=lambda x: x not in range(0,3001))

price = pd.read_csv(path + "sell_prices.csv",
                               skiprows=lambda x: x not in range(0,3001))

ss = pd.read_csv(path + "sample_submission.csv",
                               skiprows=lambda x: x not in range(0,1001))

In [4]:
display(stv.head())
display(stv.tail())
display(stv.dtypes)
display(cal.head())
display(cal.tail())
display(cal.dtypes)
display(cal.max())
display(price.head())
display(price.tail())
display(price.dtypes)
display(price.max())
display(price.shape)
display(ss.head())
display(ss.tail())
display(ss.shape)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
995,HOUSEHOLD_1_440_CA_1_validation,HOUSEHOLD_1_440,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,1,2,2,1,1,2,2,0,1,1,3,2,2,1,0,2,1,2,0,5,2,0,0,1,7,0,0,0,0,0,0,0,4,3,1,1,1,4,2,2,4,3,1,1,4,5,11,2,2,0,3,2,7,7,2,3,0,0,5,7,9,5,3,4,4,0,5,3,4,3,1,2,3,3,9,4,3,2,1,5,0,8,3,3,3,3,5,3,4,3,3,0,2,4,3,7,3,2,0,1,1,0,5,5,1,2,2,2,4,2,0,1,3,0,3,2,5,2,1,2,2,2,6,3,0,3,3,4,1,5,4,2,0,2,1,2,2,4,0,2,1,0,4,4,3,2,2,1,0,4,7,2,2,0,0,2,2,3,7,5,0,2,0,5,7,3,3,2,2,6,5,4,5,5,2,1,1,3,4,0,0,0,0,0,0,0,3,2,1,2,1,1,4,7,6,0,0,0,0,5,3,3,2,0,0,1,2,0,0,0,0,0,0,0,0,0,5,5,3,4,7,6,4,1,2,4,5,5,7,3,2,0,1,3,2,9,2,3,2,3,...,0,0,2,1,0,0,1,0,0,0,3,2,1,2,1,0,1,1,1,4,1,1,1,2,3,2,2,0,2,1,2,2,0,0,1,1,1,2,0,2,1,2,0,0,1,4,1,1,1,1,1,0,1,4,1,0,0,1,1,2,0,1,1,3,0,1,0,2,1,2,1,1,1,2,2,1,1,0,4,1,0,1,0,2,0,0,1,3,1,2,1,1,0,0,1,6,3,2,0,1,0,1,1,2,0,0,2,1,1,6,0,0,1,1,2,0,4,1,0,1,3,0,1,3,1,3,1,0,0,2,3,2,0,0,0,

id          object
item_id     object
dept_id     object
cat_id      object
store_id    object
             ...  
d_1909       int64
d_1910       int64
d_1911       int64
d_1912       int64
d_1913       int64
Length: 1919, dtype: object

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


...

store_id               CA_1
item_id       HOBBIES_1_013
wm_yr_wk              11621
sell_price             9.58
dtype: object

(3000, 4)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
995,HOUSEHOLD_1_440_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
996,HOUSEHOLD_1_441_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
997,HOUSEHOLD_1_442_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
998,HOUSEHOLD_1_443_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999,HOUSEHOLD_1_444_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(1000, 29)

In [5]:
day1_1913 = [f"d_{i}" for i in range(1, 1914)]

In [6]:
stv_melt =  pd.melt(stv, id_vars=['id','store_id','item_id'],
           value_vars=day1_1913,
           var_name = "d", value_name = "vol")

In [7]:
del day1_1913
gc.collect()

60

In [8]:
product = stv[["id","item_id","store_id"]]

In [9]:
ss_val = ss[0:30490]
ss_val.columns = ["id"] + [f"d_{d}" for d in range(1914, 1942)]

ss_eva = ss[30490:60980]
ss_eva.columns = ["id"] + [f"d_{d}" for d in range(1942, 1970)]

In [10]:
ss_eva['id'] = ss_eva['id'].str.replace('_evaluation','_validation')

In [11]:
ss_val = pd.merge(ss_val, product, how = 'left', left_on = ['id'], right_on = ['id'])
ss_eva = pd.merge(ss_eva, product, how = 'left', left_on = ['id'], right_on = ['id'])

In [12]:
display(ss_val.head(3))
display(ss_val.tail(3))
display(ss_val.shape)
display(ss_eva.head(3))
display(ss_eva.tail(3))
display(ss_eva.shape)

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,item_id,store_id
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_001,CA_1
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_002,CA_1
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_003,CA_1


,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,item_id,store_id
997,HOUSEHOLD_1_442_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOUSEHOLD_1_442,CA_1
998,HOUSEHOLD_1_443_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOUSEHOLD_1_443,CA_1
999,HOUSEHOLD_1_444_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOUSEHOLD_1_444,CA_1


(1000, 31)

,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969,id,item_id,store_id


,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969,id,item_id,store_id


(0, 31)

In [13]:
val_1914_1941 = [f"d_{i}" for i in range(1914, 1942)]
eva_1942_1969 = [f"d_{i}" for i in range(1942, 1970)]

In [14]:
val_melt =  pd.melt(ss_val, id_vars=['id','store_id', "item_id"],
           value_vars=val_1914_1941,
           var_name = "d", value_name = "vol")
eva_melt =  pd.melt(ss_eva, id_vars=['id','store_id', "item_id"],
           value_vars=eva_1942_1969,
           var_name = "d", value_name = "vol")

In [15]:
stv_melt = pd.concat([stv_melt, val_melt, eva_melt])

In [16]:
display(stv_melt.head(3))
display(stv_melt.tail(3))
display(stv_melt.shape)

,id,store_id,item_id,d,vol
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,d_1,0
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,d_1,0
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,d_1,0


,id,store_id,item_id,d,vol
27997,HOUSEHOLD_1_442_CA_1_validation,CA_1,HOUSEHOLD_1_442,d_1941,0
27998,HOUSEHOLD_1_443_CA_1_validation,CA_1,HOUSEHOLD_1_443,d_1941,0
27999,HOUSEHOLD_1_444_CA_1_validation,CA_1,HOUSEHOLD_1_444,d_1941,0


(1941000, 5)

In [17]:
del ss, ss_val, ss_eva, val_1914_1941, eva_1942_1969, val_melt, eva_melt, product
gc.collect()

20

In [18]:
cal = cal[["date","wm_yr_wk","d","event_name_1","event_type_1","event_name_2","event_type_2"]]

In [19]:
stv_melt = pd.merge(stv_melt, cal, how = 'left', left_on = ['d'], right_on = ['d'])

In [20]:
del cal
gc.collect()

60

In [21]:
display(stv_melt.head())
display(stv_melt.tail())
display(stv_melt.dtypes)
display(stv_melt.shape)

,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN


,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2
1940995,HOUSEHOLD_1_440_CA_1_validation,CA_1,HOUSEHOLD_1_440,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN
1940996,HOUSEHOLD_1_441_CA_1_validation,CA_1,HOUSEHOLD_1_441,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN
1940997,HOUSEHOLD_1_442_CA_1_validation,CA_1,HOUSEHOLD_1_442,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN
1940998,HOUSEHOLD_1_443_CA_1_validation,CA_1,HOUSEHOLD_1_443,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN
1940999,HOUSEHOLD_1_444_CA_1_validation,CA_1,HOUSEHOLD_1_444,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN


id              object
store_id        object
item_id         object
d               object
vol              int64
date            object
wm_yr_wk         int64
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
dtype: object

(1941000, 11)

In [22]:
stv_melt = stv_melt.merge(price, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [23]:
del price
gc.collect()

60

In [24]:
display(stv_melt.head())
display(stv_melt.tail())
display(stv_melt.dtypes)
display(stv_melt.shape)

,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,NaN


,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
1940995,HOUSEHOLD_1_440_CA_1_validation,CA_1,HOUSEHOLD_1_440,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN,NaN
1940996,HOUSEHOLD_1_441_CA_1_validation,CA_1,HOUSEHOLD_1_441,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN,NaN
1940997,HOUSEHOLD_1_442_CA_1_validation,CA_1,HOUSEHOLD_1_442,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN,NaN
1940998,HOUSEHOLD_1_443_CA_1_validation,CA_1,HOUSEHOLD_1_443,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN,NaN
1940999,HOUSEHOLD_1_444_CA_1_validation,CA_1,HOUSEHOLD_1_444,d_1941,0,2016-05-22,11617,NaN,NaN,NaN,NaN,NaN


id               object
store_id         object
item_id          object
d                object
vol               int64
date             object
wm_yr_wk          int64
event_name_1     object
event_type_1     object
event_name_2     object
event_type_2     object
sell_price      float64
dtype: object

(1941000, 12)

### 特徴量作成

In [25]:
stv_melt["date2"] = pd.to_datetime(stv_melt["date"])

In [26]:
stv_melt["year"] = stv_melt["date2"].dt.year
stv_melt["month"] = stv_melt["date2"].dt.month
stv_melt["week"] = stv_melt["date2"].dt.week
stv_melt["day"] = stv_melt["date2"].dt.day
stv_melt["dayofweek"] = stv_melt["date2"].dt.dayofweek

stv_melt["year"] = stv_melt["year"].astype('int8')
stv_melt["month"] = stv_melt["month"].astype('int8')
stv_melt["week"] = stv_melt["week"].astype('int8')
stv_melt["day"] = stv_melt["day"].astype('int8')
stv_melt["dayofweek"] = stv_melt["dayofweek"].astype('int8')

In [27]:
stv_melt = stv_melt.drop("date2", axis=1)

In [28]:
#　ラグの作成
for i in [7,30,90]:
    stv_melt['shift%s'%i] = stv_melt["vol"].shift(i)

In [29]:
# 平均値
'''
for i in [7,30,90]:
    stv_melt['mean%s'%i] = stv_melt["vol"].rolling(i).mean()
'''

'\nfor i in [7,30,90]:\n    stv_melt[\'mean%s\'%i] = stv_melt["vol"].rolling(i).mean()\n'

In [ ]:
# 中央値
'''
for i in [7,30,90]:
    stv_melt['median%s'%i] = stv_melt["vol"].rolling(i).median()
'''

In [ ]:
# 最小値
'''
for i in [7,30,90]:
    stv_melt['min%s'%i] = stv_melt["vol"].rolling(i).min()
'''

In [ ]:
display(stv_melt.head(3))
display(stv_melt.tail(3))
display(stv_melt.dtypes)

In [30]:
stv_melt["vol"] = stv_melt[["vol"]].astype('int16')
stv_melt["wm_yr_wk"] = stv_melt[ "wm_yr_wk"].astype('int16')

In [31]:
stv_melt["sell_price"] = stv_melt["sell_price"].astype('float16')
stv_melt["shift7"] = stv_melt["shift7"].astype('float16')

In [32]:
stv_melt["shift30"] = stv_melt["shift30"].astype('float16')
stv_melt["shift90"] = stv_melt["shift90"].astype('float16')

In [33]:
# ラベルエンコーダー
lbl = preprocessing.LabelEncoder()
stv_melt["store_id"] = lbl.fit_transform(stv_melt["store_id"])
stv_melt["item_id"] = lbl.fit_transform(stv_melt["item_id"])

stv_melt["store_id"] = stv_melt["store_id"].astype('int8')
stv_melt["item_id"] = stv_melt["item_id"].astype('int8')

In [34]:
# event は欠損値があるので前処理
stv_melt["event_name_1"] = stv_melt["event_name_1"].fillna("missing", inplace=True)
stv_melt["event_type_1"] = stv_melt["event_type_1"].fillna("missing", inplace=True)
stv_melt["event_name_2"] = stv_melt["event_name_2"].fillna("missing", inplace=True)
stv_melt["event_type_2"] = stv_melt["event_type_2"].fillna("missing", inplace=True)

stv_melt["event_name_1"] = lbl.fit_transform(stv_melt["event_name_1"])
stv_melt["event_type_1"] = lbl.fit_transform(stv_melt["event_type_1"])
stv_melt["event_name_2"] = lbl.fit_transform(stv_melt["event_name_2"])
stv_melt["event_type_2"] = lbl.fit_transform(stv_melt["event_type_2"])

stv_melt["event_name_1"] = stv_melt["event_name_1"].astype('int8')
stv_melt["event_name_2"] = stv_melt["event_name_2"].astype('int8')
stv_melt["event_type_1"] = stv_melt["event_type_1"].astype('int8')
stv_melt["event_type_2"] = stv_melt["event_type_2"].astype('int8')

In [35]:
display(stv_melt.head(3))
display(stv_melt.tail(3))
display(stv_melt.dtypes)

,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,sell_price,year,month,week,day,dayofweek,shift7,shift30,shift90
0,HOBBIES_1_001_CA_1_validation,0,0,d_1,0,2011-01-29,11101,0,0,0,0,NaN,-37,1,4,29,5,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,0,1,d_1,0,2011-01-29,11101,0,0,0,0,NaN,-37,1,4,29,5,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,0,2,d_1,0,2011-01-29,11101,0,0,0,0,NaN,-37,1,4,29,5,NaN,NaN,NaN


,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,sell_price,year,month,week,day,dayofweek,shift7,shift30,shift90
1940997,HOUSEHOLD_1_442_CA_1_validation,0,-27,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0
1940998,HOUSEHOLD_1_443_CA_1_validation,0,-26,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0
1940999,HOUSEHOLD_1_444_CA_1_validation,0,-25,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0


id               object
store_id           int8
item_id            int8
d                object
vol               int16
date             object
wm_yr_wk          int16
event_name_1       int8
event_type_1       int8
event_name_2       int8
event_type_2       int8
sell_price      float16
year               int8
month              int8
week               int8
day                int8
dayofweek          int8
shift7          float16
shift30         float16
shift90         float16
dtype: object

### 学習用データセットの作成

In [36]:
x_train = stv_melt[stv_melt['date'] <= '2016-03-27']
y_train = x_train['vol']
x_val   = stv_melt[(stv_melt['date'] > '2016-03-27') & (stv_melt['date'] <= '2016-04-24')]
y_val   = x_val['vol']
test    = stv_melt[(stv_melt['date'] > '2016-04-24')]

In [37]:
display(test.head())
display(test.tail())
display(test.dtypes)

,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,sell_price,year,month,week,day,dayofweek,shift7,shift30,shift90
1913000,HOBBIES_1_001_CA_1_validation,0,0,d_1914,0,2016-04-25,11613,0,0,0,0,8.382812,-32,4,17,25,0,1.0,1.0,1.0
1913001,HOBBIES_1_002_CA_1_validation,0,1,d_1914,0,2016-04-25,11613,0,0,0,0,3.970703,-32,4,17,25,0,0.0,4.0,5.0
1913002,HOBBIES_1_003_CA_1_validation,0,2,d_1914,0,2016-04-25,11613,0,0,0,0,2.970703,-32,4,17,25,0,1.0,0.0,0.0
1913003,HOBBIES_1_004_CA_1_validation,0,3,d_1914,0,2016-04-25,11613,0,0,0,0,4.640625,-32,4,17,25,0,5.0,17.0,0.0
1913004,HOBBIES_1_005_CA_1_validation,0,4,d_1914,0,2016-04-25,11613,0,0,0,0,2.880859,-32,4,17,25,0,0.0,0.0,2.0


,id,store_id,item_id,d,vol,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,sell_price,year,month,week,day,dayofweek,shift7,shift30,shift90
1940995,HOUSEHOLD_1_440_CA_1_validation,0,-29,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0
1940996,HOUSEHOLD_1_441_CA_1_validation,0,-28,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0
1940997,HOUSEHOLD_1_442_CA_1_validation,0,-27,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0
1940998,HOUSEHOLD_1_443_CA_1_validation,0,-26,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0
1940999,HOUSEHOLD_1_444_CA_1_validation,0,-25,d_1941,0,2016-05-22,11617,0,0,0,0,NaN,-32,5,20,22,6,0.0,0.0,0.0


id               object
store_id           int8
item_id            int8
d                object
vol               int16
date             object
wm_yr_wk          int16
event_name_1       int8
event_type_1       int8
event_name_2       int8
event_type_2       int8
sell_price      float16
year               int8
month              int8
week               int8
day                int8
dayofweek          int8
shift7          float16
shift30         float16
shift90         float16
dtype: object

In [38]:
del stv_melt
gc.collect()

120

In [39]:
features = [
    "store_id",
    "item_id",
    "sell_price",
    "shift7",
    "shift30",
    "shift90",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "year",
    "month",
    "week",
    "day",
    "dayofweek"
]

In [40]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1,
    'bagging_fraction': 0.75,
    'bagging_freq': 10, 
    'colsample_bytree': 0.75
}

### XGboost モデルの実行

In [42]:
# XGBoost が扱うデータセットの形式に直す
dtrain = xgb.DMatrix(x_train[features], label=y_train)
# dtest = xgb.DMatrix(test) # label=y_test)

# 学習用のパラメータ
xgb_params = {
    # 二値分類問題
    'objective': 'binary:logistic',
    # 評価指標
    'eval_metric': 'logloss',
}

model3 = xgb.XGBRegressor(xgb_params)

model3.fit(x_train[features], y_train)

'''
# モデルを学習する
bst = xgb.train(xgb_params,
                dtrain,
                num_boost_round=100,  # 学習ラウンド数は適当
                )

'''

# 検証用データが各クラスに分類される確率を計算する
y_pred3 = model3.predict(test[features])

XGBoostError: [19:19:13] /tmp/pip-install-ow0g0ye9/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/objective/objective.cc:26: Unknown objective function: `{'objective': 'binary:logistic', 'eval_metric': 'logloss'}`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: rank:pairwise
Objective candidate: rank:ndcg
Objective candidate: rank:map
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: count:poisson
Objective candidate: survival:cox
Objective candidate: reg:gamma
Objective candidate: reg:tweedie

Stack trace:
  [bt] (0) /home/kojismz/.local/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x1d9efe) [0x7b9ce0507efe]
  [bt] (1) /home/kojismz/.local/lib/python3.7/site-packages/xgboost/li

In [ ]:
display(y_pred3.head())
display(y_pred3.tail())
display(y_pred3.dtypes)

### しきい値 0.5 で 0, 1 に丸める
y_pred = np.where(y_pred_proba > 0.5, 1, 0)

### lightGBM モデルの実行

In [ ]:
train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

In [ ]:
model_lgb = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [train_set, val_set], verbose_eval = 100)

In [ ]:
val_pred = model_lgb.predict(x_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score は {val_score}')

In [ ]:
y_pred = model_lgb.predict(test[features])
test['vol'] = y_pred

In [ ]:
predictions = test[['id', 'date', 'vol']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'vol').reset_index()

In [ ]:
display(predictions.head())
display(predictions.tail())
display(predictions.shape)

In [ ]:
del features, params, x_train, y_train, x_val, y_val, test, model_lgb
gc.collect()

In [ ]:
pre_val = predictions.iloc[:,:29]

In [ ]:
pre_eva = pd.concat([predictions.iloc[:,0],predictions.iloc[:,29:57]], axis=1)
pre_eva['id'] = pre_eva['id'].str.replace('_validation', '_evaluation')

In [ ]:
del predictions
gc.collect()

In [ ]:
display(pre_val.head())
display(pre_val.tail())
display(pre_val.shape)

display(pre_eva.head())
display(pre_eva.tail())
display(pre_eva.shape)

In [ ]:
pre_val.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
pre_eva.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

### catboostの実行

In [ ]:
# initialize Pool
train_pool = Pool(x_train[features], 
                  y_train)

test_pool = Pool(test[features]) 

# specify the training parameters
model2 = CatBoostRegressor(iterations=2000,
                          depth=5,
                          learning_rate=0.05,
                          loss_function='RMSE')
#train the model
model2.fit(train_pool)
# make the prediction using the resulting model
preds2 = model2.predict(test_pool)

In [ ]:
test2 = test
test2['vol'] = preds2

In [ ]:
predictions2 = test2[['id', 'date', 'vol']]
predictions2 = pd.pivot(predictions2, index = 'id', columns = 'date', values = 'vol').reset_index()

In [ ]:
pre_val2 = predictions2.iloc[:,:29]

In [ ]:
pre_eva2 = pd.concat([predictions2.iloc[:,0],predictions2.iloc[:,29:57]], axis=1)
pre_eva2['id'] = pre_eva2['id'].str.replace('_validation', '_evaluation')

In [ ]:
pre_val2.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
pre_eva2.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [ ]:
display(pre_val2.head())
display(pre_val2.tail())
display(pre_val2.shape)

display(pre_eva2.head())
display(pre_eva2.tail())
display(pre_eva2.shape)

### 誤差率の検証

In [ ]:
sta = pd.read_csv(path + "sales_train_evaluation.csv")
sta = sta[["d_1914", "d_1915","d_1916","d_1917","d_1918","d_1919","d_1920","d_1921","d_1922","d_1923","d_1924","d_1925","d_1926","d_1927","d_1928","d_1929","d_1930","d_1931","d_1932","d_1933","d_1934","d_1935","d_1936","d_1937","d_1938","d_1939","d_1940","d_1941"]]
sta.columns = ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]

In [ ]:
for i in ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]:
    sta[i] = sta[i].astype('float64')

In [ ]:
pre_val_temp = pre_val.drop("id",axis=1)
pre_val2_temp = pre_val2.drop("id",axis=1)

In [ ]:
display(sta.head())
display(sta.tail())
display(sta.dtypes)
display(pre_val_temp.head())
display(pre_val_temp.tail())
display(pre_val_temp.dtypes)

### d_1914_1941 の誤差率

In [ ]:
# 二乗平均平方根誤差 (RMSE)
display(np.sqrt(mean_squared_error(sta, pre_val_temp)))
display(np.sqrt(mean_squared_error(sta, pre_val2_temp)))

In [ ]:
# 平均絶対誤差 (MAE)
display(mean_absolute_error(sta, pre_val_temp))
display(mean_absolute_error(sta, pre_val2_temp))

In [ ]:
# 決定係数
# モデルの当てはまりの良さを示す指標で、最も当てはまりの良い場合、1.0 となります
display(r2_score(sta, pre_val_temp))
display(r2_score(sta, pre_val2_temp))

del sta, pre_val_temp,pre_val2_temp, i
gc.collect()

# データ統合 csv保存

# lightGBM

In [ ]:
pre_uni = pd.concat([pre_val, pre_eva], axis=0)

# catboost

In [ ]:
pre_uni2 = pd.concat([pre_val2, pre_eva2], axis=0)

In [ ]:
del pre_val, pre_eva, pre_val2, pre_eva2
gc.collect()

In [ ]:
pre_uni_am = pre_uni.set_index('id') * 0.4 + pre_uni2.set_index('id') * 0.6

In [ ]:
pre_uni_am = pre_uni_am.reset_index()

In [ ]:
del pre_uni, pre_uni2
gc.collect()

In [ ]:
display(pre_uni_am.head())
display(pre_uni_am.tail())
display(pre_uni_am.shape)

In [ ]:
pre_uni_am.to_csv('submission_uni.csv', index = False)

In [ ]:
del pre_uni_am
gc.collect()